In [28]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import ultralytics
from ultralytics import YOLO
from PIL import Image
import os
import numpy as np
import math

In [2]:
# model_path = "D:/Skripsi/Results/9-11-2023-50epch/weights/best.pt"
# model = YOLO()
# checkpoint = torch.load(model_path)
# print(checkpoint.keys())

In [38]:
# Load the model
model_path = "D:/Skripsi/Results/9-11-2023-50epch/weights/best.pt"
model = YOLO()
checkpoint = torch.load(model_path)
model = checkpoint['model']
model = model.to(torch.float32)  # Convert model to full precision
model.eval()

class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_name = self.images[idx]
        image_path = os.path.join(self.image_dir, image_name)
        label_path = os.path.join(self.label_dir, image_name.replace('.jpg', '.txt'))
        
        image = Image.open(image_path).convert("RGB")
        label = self.read_label(label_path)
        
        if self.transform:
            image = self.transform(image)

        return image, label, image_path

    def read_label(self, label_path):
        with open(label_path, 'r') as file:
            labels = [int(line.split()[0]) for line in file.readlines()]
            
            return labels
        pass

def process_yolo_outputs(outputs, confidence_threshold=0.5):
    processed_predictions = []

    for output in outputs:  # output could be a tensor or a list of tensors
        # Check if output is a tensor
        if isinstance(output, torch.Tensor):
            # Flatten the output if it's a batch of predictions
            if output.ndim > 2:
                output = output.view(-1, output.size(-1))
            output = [output]
        
        for tensor in output:  # Process each tensor
            for pred in tensor:
                object_confidence = pred[4]  # The object confidence score

                # Check if object_confidence is a single value
                if object_confidence.numel() == 1 and object_confidence.item() > confidence_threshold:
                    class_probabilities = pred[5:]  # Class probabilities
                    class_prediction = torch.argmax(class_probabilities).item()
                    processed_predictions.append(class_prediction)

    return processed_predictions


# Define transforms
transform = transforms.Compose([
    transforms.Resize((416, 416)),  # Resize images to fit YOLO input
    transforms.ToTensor()
])

# Load test dataset (now validation dataset)
val_dataset = CustomDataset('D:/Skripsi/dataset/images/val', 'D:/Skripsi/dataset/labels/val', transform=transform)
val_loader = DataLoader(val_dataset, shuffle=False)

all_predictions = []
all_labels = []

for images, yolo_labels, paths in tqdm(val_loader, desc="Processing Validation Dataset"):
    with torch.no_grad():
        outputs = model(images)
        predictions = process_yolo_outputs(outputs)

    all_predictions.extend(predictions)
    all_labels.extend(yolo_labels)

Processing Validation Dataset:  63%|███████████████████████████                | 20278/32153 [1:41:52<59:39,  3.32it/s]


KeyboardInterrupt: 

In [37]:
def calculate_classification_metrics(predictions, labels):
    # Calculate precision, recall, and F1-score
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    f1 = f1_score(labels, predictions, average='macro')
    
    return precision, recall, f1

# Calculate precision, recall, and F1-score
precision, recall, f1 = calculate_classification_metrics(all_predictions, all_labels)

# Print the metrics
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

ValueError: Found input variables with inconsistent numbers of samples: [5, 128612]